# Создание нейронки для преобразования неструктурированных описаний пшеницы в структурированные

In [ ]:
!pip install datasets
import pandas as pd
import json
import string
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
file_path = r"/content/stack.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

df = pd.DataFrame(data)

df.head()

,название,Культура,Группа,Код сорта,Описание,Автор(ы),Номер заявки на допуск,Заявители,Дата регистрации заявки на допуск,Год включения в реестр допущенных,...,Сумма пошлины за текущий (9) год поддержания,Сумма пошлины за текущий (8) год поддержания,Сумма пошлины за текущий (28) год поддержания,Сумма пошлины за текущий (14) год поддержания,Сумма пошлины за текущий (22) год поддержания,Сумма пошлины за текущий (23) год поддержания,Сумма пошлины за текущий (19) год поддержания,Сумма пошлины за текущий (26) год поддержания,Сумма пошлины за текущий (20) год поддержания,Сумма пошлины за текущий (27) год поддержания
0,100 ЛЕТ ТАССР,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,8058807,Родословная- (Радуга х О-152/03-7(2) х (Радуга...,АСХАДУЛЛИН ДАМИР ФИДУСОВИЧ; АСХАДУЛЛИН ДАНИЛ Ф...,82178,ФГБУН 'ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР 'КА...,03 ноября 2020г.,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,АВИАДА,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,9809918,Родословная- внутривидовая гибридизация озимог...,ЛЕОНОВА ТАТЬЯНА АЛЕКСЕЕВНА; НОВОХАТИН ВЛАДИМИР...,35123,ФГБУН ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР ТЮМЕ...,09 января 2001г.,2004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,АГАТА,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,8853479,Родословная- Артемовка х Мильтурум 63. Включен...,АНТОШИНА ОЛЬГА АЛЕКСЕЕВНА; БАРКОВСКАЯ ТАТЬЯНА ...,56903,ФГБНУ 'ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР 'НЕ...,10 ноября 2011г.,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,АГРОНОМИЧЕСКАЯ 5,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,7953809,"Родословная- создан с участием сортов Соната, ...",АЙДАРОВ АМАНЖОЛ НУРЖАН УЛЫ; ГЛАДКИХ МАРИНА СЕР...,82007,ФГБОУ ВО 'ОМСКИЙ ГОСУДАРСТВЕННЫЙ АГРАРНЫЙ УНИВ...,23 октября 2020г.,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,АГРОС,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,7852652,Родословная- Злата x Московская 56. Включен в ...,ВОРОНОВ СЕРГЕЙ ИВАНОВИЧ; ДАВЫДОВА НАТАЛЬЯ ВЛАД...,84044,ООО 'НАДЕЖДА' Г. МОСКВА,05 июля 2021г.,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re


df_desc_not_str = df['Описание'].copy()
df_desc_not_str = pd.DataFrame(df_desc_not_str)
df_desc_not_str = df_desc_not_str.dropna()
df_desc_not_str = df_desc_not_str.reset_index(drop=True)

df_desc_not_str.head()

,Описание
0,Родословная- (Радуга х О-152/03-7(2) х (Радуга...
1,Родословная- внутривидовая гибридизация озимог...
2,Родословная- Артемовка х Мильтурум 63. Включен...
3,"Родословная- создан с участием сортов Соната, ..."
4,Родословная- Злата x Московская 56. Включен в ...


In [ ]:
import pandas as pd

df_desc_str = pd.DataFrame(columns=['Описание'])

file_path = r"/content/structured_with_pedigree.txt"

with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

descriptions = []
current_description = []

for line in lines:
    stripped_line = line.strip()
    if stripped_line.startswith("Описание"):
        if current_description:
            descriptions.append({'Описание': " ".join(current_description)})
            current_description = []
    else:
        current_description.append(stripped_line)

if current_description:
    descriptions.append({'Описание': " ".join(current_description)})

df_desc_str = pd.DataFrame(descriptions)

df_desc_str.head()

,Описание
0,Родословная: (Радуга х О-152/03-7(2) х (Радуга...
1,Родословная: внутривидовая гибридизация озимог...
2,Родословная: Артемовка х Мильтурум 63 Госреест...
3,"Родословная: создан с участием сортов Соната, ..."
4,Родословная: Злата x Московская 56 Госреестр: ...


In [ ]:
df_desc_str = df_desc_str.rename(columns={'Описание': 'structured'})
df_desc_not_str = df_desc_not_str.rename(columns={'Описание': 'unstructured'})

In [ ]:
df_concat = pd.concat([df_desc_str, df_desc_not_str], axis=1)


In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_concat['unstructured_clean'] = df_concat['unstructured'].apply(clean_text)
df_concat['structured_clean'] = df_concat['structured'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())

In [ ]:
data_dict = {
    "input_text": df_concat["unstructured_clean"].tolist(),
    "target_text": df_concat["structured_clean"].tolist()
}


In [ ]:
dataset = Dataset.from_dict(data_dict)
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
task_prefix = "generate structured description: "

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [ ]:
def preprocess_function(examples):
    inputs = [task_prefix + ex for ex in examples["input_text"]]
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True) # Add padding='max_length'
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=512, padding='max_length', truncation=True) # Add padding='max_length'
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    save_strategy="epoch"

)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
trainer.train()